In [1]:
import openai

In [57]:
import pandas as pd
import numpy 
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pickle
import tiktoken

In [2]:
from dotenv import dotenv_values

In [3]:
config = dotenv_values(".env")


In [4]:
openai.api_key = config['OPENAI_API_KEY']

In [5]:
res = openai.Embedding.create(
    input='candy canes',
    model = "text-embedding-ada-002"

)

In [7]:
res["data"][0]["embedding"]

[-0.01746266707777977,
 -0.023939916864037514,
 0.004187542479485273,
 -0.03715350851416588,
 -0.025831274688243866,
 0.0006748485029675066,
 -0.018175164237618446,
 -0.007824518717825413,
 -0.00906167272478342,
 -0.029847169294953346,
 0.020662428811192513,
 0.018978342413902283,
 -0.03808623179793358,
 0.004411007277667522,
 -0.0008137045660987496,
 0.005767991300672293,
 0.03904486447572708,
 0.0031641367822885513,
 -0.008187244646251202,
 -0.019936976954340935,
 -0.01642630621790886,
 -0.005240095313638449,
 -0.005246572662144899,
 -0.016387443989515305,
 0.007662586867809296,
 -0.01607653498649597,
 0.021154699847102165,
 -0.006898271385580301,
 0.003568965010344982,
 0.006775203626602888,
 0.014975402504205704,
 -0.013291317969560623,
 -0.03782714158296585,
 -0.02530013956129551,
 -0.012021776288747787,
 -0.01764402911067009,
 -0.0070213391445577145,
 -0.01652994193136692,
 0.013835406862199306,
 0.0026459568180143833,
 -0.0011424249969422817,
 -0.00319814239628613,
 -0.006386568

In [11]:
dataset_path = "./wiki_movie_plots_deduped.csv"
df = pd.read_csv(dataset_path)

In [12]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [24]:
am_movies = df[df["Origin/Ethnicity"] == "American"].sort_values("Release Year", ascending = False).head(5000)

In [25]:
am_movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."
17243,2017,"Everything, Everything",American,Stella Meghie,Stella Meghie (director); J. Mills Goodloe (sc...,"romance, drama","https://en.wikipedia.org/wiki/Everything,_Ever...","Eighteen-year-old Maddy suffers from SCID, an ..."
17241,2017,Alien: Covenant,American,Ridley Scott,"Ridley Scott (director); Michael Green, Jack P...","sci-fi, horror",https://en.wikipedia.org/wiki/Alien:_Covenant,"In a prologue, business magnate Peter Weyland ..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
...,...,...,...,...,...,...,...,...
12478,1995,Jury Duty,American,John Fortenberry,"Pauly Shore, Tia Carrere, Abe Vigoda",comedy,https://en.wikipedia.org/wiki/Jury_Duty_(film),Tommy Collins is an unemployed erotic dancer l...
12506,1995,Mortal Kombat,American,Paul W. S. Anderson,"Robin Shou, Linden Ashby, Bridgette Wilson, Ch...",fantasy,https://en.wikipedia.org/wiki/Mortal_Kombat_(f...,Mortal Kombat is a fighting tournament between...
12480,1995,A Kid in King Arthur's Court,American,Michael Gottlieb,"Thomas Ian Nicholas, Joss Ackland",comedy,https://en.wikipedia.org/wiki/A_Kid_in_King_Ar...,Calvin Fuller is a nerdy young boy living in t...
12508,1995,Mr. Holland's Opus,American,Stephen Herek,"Richard Dreyfuss, Glenne Headly, Jay Thomas, O...",drama,https://en.wikipedia.org/wiki/Mr._Holland%27s_...,"In Portland, Oregon in 1965, Glenn Holland is ..."


In [16]:
df["Origin/Ethnicity"].unique()

array(['American', 'Australian', 'Bangladeshi', 'British', 'Canadian',
       'Chinese', 'Egyptian', 'Hong Kong', 'Filipino', 'Assamese',
       'Bengali', 'Bollywood', 'Kannada', 'Malayalam', 'Marathi',
       'Punjabi', 'Tamil', 'Telugu', 'Japanese', 'Malaysian', 'Maldivian',
       'Russian', 'South_Korean', 'Turkish'], dtype=object)

In [26]:
british = df[df["Origin/Ethnicity"] == "British"]

In [31]:
british.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
18040,1920,The Amateur Gentleman,British,Maurice Elvey,"Langhorn Burton, Cecil Humphreys",drama,https://en.wikipedia.org/wiki/The_Amateur_Gent...,In Regency Britain a young man tries to establ...
18041,1920,Anna the Adventuress,British,Cecil Hepworth,"Alma Taylor, Jean Cadell, James Carew",crime,https://en.wikipedia.org/wiki/Anna_the_Adventu...,Two identical sisters are able to switch place...
18042,1920,At the Villa Rose,British,Maurice Elvey,"Manora Thew, Langhorn Burton",crime,https://en.wikipedia.org/wiki/At_the_Villa_Ros...,Inspector Hanaud is asked to investigate a mur...
18043,1920,Darby and Joan,British,Percy Nash,"Derwent Hall Caine, Ivy Close",drama,https://en.wikipedia.org/wiki/Darby_and_Joan_(...,Mrs. Gorry and her son lived by an old mill st...
18044,1920,General Post,British,Thomas Bentley,"Douglas Munro, Lilian Braithwaite",drama,https://en.wikipedia.org/wiki/General_Post,During the First World War a small-town tailor...


In [36]:
filtered_rows = df[df["Origin/Ethnicity"].isin(["American", "British"])]

In [37]:
filtered_rows

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
21705,2017,Murder on the Orient Express,British,Directors: Kenneth Branagh,Directors: Kenneth Branagh\r\nCast: Kenneth Br...,unknown,https://en.wikipedia.org/wiki/Murder_on_the_Or...,"In 1934, famous Belgian detective Hercule Poir..."
21706,2017,Paddington 2,British,Directors: Paul King,"Directors: Paul King\r\nCast: Ben Whishaw, Hug...",unknown,https://en.wikipedia.org/wiki/Paddington_2,"Paddington, having settled with the Brown fami..."
21707,2017,Finding Your Feet,British,Directors: Richard Loncraine,Directors: Richard Loncraine\r\nCast: Imelda S...,unknown,https://en.wikipedia.org/wiki/Finding_Your_Feet,‘Lady’ Sandra Abbott (Imelda Staunton) discove...
21708,2017,All the Money in the World,British,Director: Ridley Scott,Director: Ridley Scott\r\nCast: Michelle Willi...,unknown,https://en.wikipedia.org/wiki/All_the_Money_in...,"In 1973, 16-year-old John Paul Getty III (Paul..."


In [40]:
movies=filtered_rows.sort_values("Release Year", ascending = False)

In [41]:
movies.head(5000)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
21709,2017,"You, Me and Him",British,Daisy Aitkens,NaN,unknown,"https://en.wikipedia.org/wiki/You,_Me_and_Him_...","Olivia, a career lawyer in her 40's, feels rea..."
17242,2017,Diary of a Wimpy Kid: The Long Haul,American,David Bowers,"David Bowers (director); Adam Sztykiel, Brett ...","comedy, family",https://en.wikipedia.org/wiki/Diary_of_a_Wimpy...,"One year after the previous film, while at the..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
17238,2017,Lowriders,American,Ricardo de Montreuil,Ricardo de Montreuil (director); Justin Tippin...,drama,https://en.wikipedia.org/wiki/Lowriders_(film),A contemporary tale about a teen whose loyalti...
...,...,...,...,...,...,...,...,...
13371,1998,Stepmom,American,Chris Columbus,"Julia Roberts, Susan Sarandon, Ed Harris",drama,https://en.wikipedia.org/wiki/Stepmom_(film),Jackie and Luke Harrison are a divorced New Yo...
13372,1998,Strangeland,American,John Pieplow,"Kevin Gage, Robert Englund, Amy Smart",horror,https://en.wikipedia.org/wiki/Strangeland_(film),15-year-old Genevieve Gage and her best friend...
13373,1998,Suicide Kings,American,Peter O'Fallon,"Christopher Walken, Denis Leary, Sean Patrick ...",crime,https://en.wikipedia.org/wiki/Suicide_Kings,Charlie Barret walks to his private table in a...
13374,1998,Tarzan and the Lost City,American,Carl Schenkel,"Casper Van Dien, Jane March, Ian Roberts",adventure,https://en.wikipedia.org/wiki/Tarzan_and_the_L...,"In 1913, on the night before Jane Porter's wed..."


In [67]:
movies = df[df["Origin/Ethnicity"]=="American"].sort_values("Release Year", ascending=False).head(5000)

In [68]:
movie_plots = movies["Plot"].values 

## Generating Embeddings


In [47]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):
    #replace newlines, which can negatively affect performance 
    text = text.replace("\n"," ")

    return openai.Embedding.create(input=text, model=model)['data'][0]["embedding"]

In [60]:
enc = tiktoken.encoding_for_model("text-embedding-ada-002")

In [61]:
len(enc.encode("asldfj"))

4

In [69]:
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])

In [71]:
total_tokens
cost = total_tokens * (.0004/1000)


In [72]:
print(f"estimated cost ${cost:.2f}")

estimated cost $1.45


In [53]:
#establish a cache of embeddings to avoid recomputing 
#cache is a dict of tuples (text, model) -> embedding, saved as pickle file 

#set path to a embedding cahce 
embedding_cache_path = "movie_embeddings_cache.pkl"

#load the cache if it exists and ave a copy to disk 
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path,"wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

#define a function to retrieve embeddings from the cache if present and otherwise
# request via the API 

# def embedding_from_string(
#     string,
#     model="text-embedding-ada-002",
#     embedding_cache: embedding_cache
# ):
def embedding_from_string(
    string,
    model="text-embedding-ada-002",
    embedding_cache=embedding_cache
):
    ''' Return embedding of given string using a chace to avoid recomputing''' 
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string,model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPENAI FOR {string[:20]}")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string,model)]

In [84]:
# embedding_from_string("I like coffee")

In [97]:
movies_1990

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."
17243,2017,"Everything, Everything",American,Stella Meghie,Stella Meghie (director); J. Mills Goodloe (sc...,"romance, drama","https://en.wikipedia.org/wiki/Everything,_Ever...","Eighteen-year-old Maddy suffers from SCID, an ..."
17241,2017,Alien: Covenant,American,Ridley Scott,"Ridley Scott (director); Michael Green, Jack P...","sci-fi, horror",https://en.wikipedia.org/wiki/Alien:_Covenant,"In a prologue, business magnate Peter Weyland ..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
...,...,...,...,...,...,...,...,...
13716,2000,Quills,American,Philip Kaufman,"Geoffrey Rush, Kate Winslet, Joaquin Phoenix, ...","drama, biography",https://en.wikipedia.org/wiki/Quills,Quills begins in Paris during the Reign of Ter...
13717,2000,Ready to Rumble,American,Brian Robbins,"David Arquette, Oliver Platt, Scott Caan",comedy,https://en.wikipedia.org/wiki/Ready_to_Rumble,"For most of their lives, Sewage workers Gordie..."
13715,2000,Psycho Beach Party,American,Robert Lee King,"Thomas Gibson, Lauren Ambrose, Charles Busch",comedy,https://en.wikipedia.org/wiki/Psycho_Beach_Party,Florence Forrest (Lauren Ambrose) is a Gidget-...
13719,2000,Reindeer Games,American,John Frankenheimer,"Ben Affleck, Gary Sinise, Charlize Theron",crime drama,https://en.wikipedia.org/wiki/Reindeer_Games,Nick Cassidy and Rudy Duncan are cellmates in ...


In [132]:
# british = df[df["Origin/Ethnicity"] == "British"]
movies_2014= movies[movies["Release Year"] > 2014]

In [133]:
movies_2014

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."
17243,2017,"Everything, Everything",American,Stella Meghie,Stella Meghie (director); J. Mills Goodloe (sc...,"romance, drama","https://en.wikipedia.org/wiki/Everything,_Ever...","Eighteen-year-old Maddy suffers from SCID, an ..."
17241,2017,Alien: Covenant,American,Ridley Scott,"Ridley Scott (director); Michael Green, Jack P...","sci-fi, horror",https://en.wikipedia.org/wiki/Alien:_Covenant,"In a prologue, business magnate Peter Weyland ..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
...,...,...,...,...,...,...,...,...
16988,2015,Pitch Perfect 2,American,Elizabeth Banks,Anna Kendrick\r\nSkylar Astin\r\nRebel Wilson\...,comedy,https://en.wikipedia.org/wiki/Pitch_Perfect_2,Three years after winning the previous competi...
16987,2015,Mad Max: Fury Road,American,George Miller,Tom Hardy\r\nCharlize Theron\r\nNicholas Hoult...,action,https://en.wikipedia.org/wiki/Mad_Max:_Fury_Road,"Following a nuclear holocaust, the world has b..."
16986,2015,Maggie,American,Henry Hobson,Abigail Breslin\r\nArnold Schwarzenegger\r\nDe...,drama horror thriller film,https://en.wikipedia.org/wiki/Maggie_(film),"In the present-day Midwestern United States, s..."
16985,2015,Hot Pursuit,American,Anne Fletcher,Reese Witherspoon\r\nSofía Vergara\r\n,comedy,https://en.wikipedia.org/wiki/Hot_Pursuit_(201...,Rose Cooper is a San Antonio Police Department...


In [134]:
movie_plots = movies_2014["Plot"].values 

In [135]:
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])

In [136]:
total_tokens
cost = total_tokens * (.0004/1000)
print(f"estimated cost ${cost:.2f}")

estimated cost $0.15


In [137]:
total_tokens

370940

In [138]:
# This line actaully generates the embeddings
plot_embeddings = [embedding_from_string(plot, model="text-embedding-ada-002") for plot in movie_plots]

GOT EMBEDDING FROM OPENAI FOR In 1954 London, reno
GOT EMBEDDING FROM OPENAI FOR Eighteen-year-old Ma
GOT EMBEDDING FROM OPENAI FOR In a prologue, busin
GOT EMBEDDING FROM OPENAI FOR Anne (Diane Lane) is
GOT EMBEDDING FROM OPENAI FOR During the Iraq War,
GOT EMBEDDING FROM OPENAI FOR A contemporary tale 
GOT EMBEDDING FROM OPENAI FOR Recently fired from 
GOT EMBEDDING FROM OPENAI FOR The movie opens with
GOT EMBEDDING FROM OPENAI FOR Mary (Debra Winger) 
GOT EMBEDDING FROM OPENAI FOR In 2014, Peter Quill
GOT EMBEDDING FROM OPENAI FOR A young street magic
GOT EMBEDDING FROM OPENAI FOR Having made a career
GOT EMBEDDING FROM OPENAI FOR When her car breaks 
GOT EMBEDDING FROM OPENAI FOR Mikael (Oscar Isaac)
GOT EMBEDDING FROM OPENAI FOR Julia Banks is being
GOT EMBEDDING FROM OPENAI FOR This documentary fol
GOT EMBEDDING FROM OPENAI FOR Best friends Mindy (
GOT EMBEDDING FROM OPENAI FOR Dash (voiced by Schw
GOT EMBEDDING FROM OPENAI FOR Ireland, 1905: Percy
GOT EMBEDDING FROM OPENAI FOR T

In [139]:
plot_embeddings[0]

[-0.0008735599694773555,
 -0.018797315657138824,
 -0.02078908495604992,
 0.0015301347011700273,
 -0.004004284739494324,
 0.014882938005030155,
 0.0042912932112813,
 -0.0040596118196845055,
 -0.024191688746213913,
 -0.025574862957000732,
 -0.0012941309250891209,
 0.0014004623517394066,
 0.01195061206817627,
 -0.018105730414390564,
 -0.011501080356538296,
 -0.0011514911893755198,
 0.027414482086896896,
 0.010830242186784744,
 0.011563323438167572,
 -0.029323259368538857,
 -0.011030801571905613,
 0.012960327789187431,
 0.0069677322171628475,
 -0.01211659237742424,
 -0.009301835671067238,
 0.0017332881689071655,
 0.021881790831685066,
 -0.01557452417910099,
 0.01539471186697483,
 -0.013997707515954971,
 0.0028112984728068113,
 -0.0014341771602630615,
 -0.003516716416925192,
 -0.013008738867938519,
 -0.03947574645280838,
 -0.03048512525856495,
 0.005335588473826647,
 -0.010996222496032715,
 0.017096014693379402,
 0.014025370590388775,
 0.005919978953897953,
 0.00524222431704402,
 -0.0212870

In [140]:
len(plot_embeddings)

467

In [145]:
data = movies_2014[["Title", "Genre"]].to_dict("records")

In [141]:
from nomic import atlas

In [146]:
project = atlas.map_embeddings(
    embeddings = numpy.array(plot_embeddings),
    data = data
)

2023-04-27 19:58:31.729 | INFO     | nomic.project:_create_project:965 - Creating project `sloppy-fetus` in organization `tajatheprince`
2023-04-27 19:58:32.810 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
1it [00:01,  1.26s/it]
2023-04-27 19:58:34.084 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-04-27 19:58:34.084 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-04-27 19:58:34.852 | INFO     | nomic.project:create_index:1282 - Created map `sloppy-fetus` in project `sloppy-fetus`: https://atlas.nomic.ai/map/74375f42-fc37-40e3-8c10-785d955b8a9f/3ce3eb53-ea29-4224-aa59-0b9104d953e3
2023-04-27 19:58:34.853 | INFO     | nomic.atlas:map_embeddings:132 - sloppy-fetus: https://atlas.nomic.ai/map/74375f42-fc37-40e3-8c10-785d955b8a9f/3ce3eb53-ea29-4224-aa59-0b9104d953e3


In [162]:
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

In [173]:
def print_recommendations_from_strings(
    strings,
    index_of_source_string,
    k_nearest_neighbors=3,
    model = "text-embedding-ada-002"
):
    

    #get all of the embeddings
    embeddings = [embedding_from_string(string) for string in strings]
    # get embedding for our specific query string
    query_embedding = embeddings[index_of_source_string]
    # get disntances betweenour embedding and other embeddings
    distances = distances_from_embeddings(query_embedding, embeddings) 
    # print(distances) 
    
    # get indices of the nearest neighbors 
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
    query_string = strings[index_of_source_string]
    match_count = 0
    for i in indices_of_nearest_neighbors:
        if query_string == strings[i]:
            continue 
        if match_count >= k_nearest_neighbors:
            break 
        match_count += 1
        print(f"found{match_count} closest match: ")
        print(f"distance of: {distances[i]}")
        print(strings[i])

    

In [174]:
print_recommendations_from_strings(movie_plots, 2)

found1 closest match: 
distance of: 0.1365166895341312
In the near future, the unmanned Pilgrim 7 space probe returns from Mars to Earth orbit with soil samples potentially containing evidence of extraterrestrial life. The probe is captured and its samples retrieved by the International Space Station and its six-member crew. Exobiologist Hugh Derry, who is paralyzed from the waist down, revives a dormant cell from the sample, which quickly grows into a multi-celled organism that American school children name "Calvin". Hugh realizes that Calvin's cells can change their specialisation, acting as muscle, sensor, and neuron cells all at once.
An accident in the lab causes Calvin to become dormant; Hugh attempts to revive Calvin with electric shocks, but Calvin immediately becomes hostile and attacks Hugh, crushing his hand. While Hugh lies unconscious from Calvin's attack, Calvin uses Hugh's electric shock tool to escape its enclosure; now free in the laboratory, Calvin devours a lab rat b

In [171]:
movies_2014

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."
17243,2017,"Everything, Everything",American,Stella Meghie,Stella Meghie (director); J. Mills Goodloe (sc...,"romance, drama","https://en.wikipedia.org/wiki/Everything,_Ever...","Eighteen-year-old Maddy suffers from SCID, an ..."
17241,2017,Alien: Covenant,American,Ridley Scott,"Ridley Scott (director); Michael Green, Jack P...","sci-fi, horror",https://en.wikipedia.org/wiki/Alien:_Covenant,"In a prologue, business magnate Peter Weyland ..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
...,...,...,...,...,...,...,...,...
16988,2015,Pitch Perfect 2,American,Elizabeth Banks,Anna Kendrick\r\nSkylar Astin\r\nRebel Wilson\...,comedy,https://en.wikipedia.org/wiki/Pitch_Perfect_2,Three years after winning the previous competi...
16987,2015,Mad Max: Fury Road,American,George Miller,Tom Hardy\r\nCharlize Theron\r\nNicholas Hoult...,action,https://en.wikipedia.org/wiki/Mad_Max:_Fury_Road,"Following a nuclear holocaust, the world has b..."
16986,2015,Maggie,American,Henry Hobson,Abigail Breslin\r\nArnold Schwarzenegger\r\nDe...,drama horror thriller film,https://en.wikipedia.org/wiki/Maggie_(film),"In the present-day Midwestern United States, s..."
16985,2015,Hot Pursuit,American,Anne Fletcher,Reese Witherspoon\r\nSofía Vergara\r\n,comedy,https://en.wikipedia.org/wiki/Hot_Pursuit_(201...,Rose Cooper is a San Antonio Police Department...


In [176]:
def print_recommendations_from_title(
    title_to_plot_dict,
    source_title,
    k_nearest_neighbors=3,
    model="text-embedding-ada-002"
):
    # Make sure the source_title is in the dictionary
    if source_title not in title_to_plot_dict:
        print("Error: Source title not found in the dictionary.")
        return

    source_string = title_to_plot_dict[source_title]

    # Get all the embeddings
    embeddings = [embedding_from_string(plot) for plot in title_to_plot_dict.values()]

    # Get the embedding for our specific source_title
    query_embedding = embedding_from_string(source_string)

    # Get distances between our embedding and other embeddings
    distances = distances_from_embeddings(query_embedding, embeddings)

    # Get indices of the nearest neighbors
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    titles = list(title_to_plot_dict.keys())
    match_count = 0
    for i in indices_of_nearest_neighbors:
        if source_title == titles[i]:
            continue
        if match_count >= k_nearest_neighbors:
            break
        match_count += 1
        print(f"found {match_count} closest match:")
        print(f"distance of: {distances[i]}")
        print(titles[i])

# Usage example:
# title_to_plot_dict = {
#     "The Godfather": "The aging patriarch of an organized crime dynasty...",
#     "The Godfather Part II": "The early life and career of Vito Corleone...",
#     "Goodfellas": "The story of Henry Hill and his life...",
#     "Pulp Fiction": "The lives of two mob hitmen...",
#     "The Shawshank Redemption": "Two imprisoned men bond over...",
#     "Schindler's List": "In German-occupied Poland...",
#     "Forrest Gump": "The presidencies of Kennedy and Johnson...",
#     "The Dark Knight": "When the menace known as the Joker...",
#     "Inception": "A thief who steals corporate secrets...",
#     "Fight Club": "An insomniac office worker...",
#     "The Matrix": "A computer hacker learns...",
# }

source_title = "Phantom Thread"
print_recommendations_from_title(movie_plots, source_title)


GOT EMBEDDING FROM OPENAI FOR The aging patriarch 
GOT EMBEDDING FROM OPENAI FOR The early life and c
GOT EMBEDDING FROM OPENAI FOR The story of Henry H
GOT EMBEDDING FROM OPENAI FOR The lives of two mob
GOT EMBEDDING FROM OPENAI FOR Two imprisoned men b
GOT EMBEDDING FROM OPENAI FOR In German-occupied P
GOT EMBEDDING FROM OPENAI FOR The presidencies of 
GOT EMBEDDING FROM OPENAI FOR When the menace know
GOT EMBEDDING FROM OPENAI FOR A thief who steals c
GOT EMBEDDING FROM OPENAI FOR An insomniac office 
GOT EMBEDDING FROM OPENAI FOR A computer hacker le
found 1 closest match:
distance of: 0.12180757607030823
The Godfather Part II
found 2 closest match:
distance of: 0.12661094360142
Pulp Fiction
found 3 closest match:
distance of: 0.1521478224808117
Inception


In [178]:
def print_recommendations_from_title(
    title_to_plot_dict,
    source_title,
    k_nearest_neighbors=3,
    model="text-embedding-ada-002"
):
    # Make sure the source_title is in the dictionary
    if source_title not in title_to_plot_dict:
        print("Error: Source title not found in the dictionary.")
        return

    source_string = title_to_plot_dict[source_title]

    # Get all the embeddings
    embeddings = [embedding_from_string(plot) for plot in title_to_plot_dict.values()]

    # Get the embedding for our specific source_title
    query_embedding = embedding_from_string(source_string)

    # Get distances between our embedding and other embeddings
    distances = distances_from_embeddings(query_embedding, embeddings)

    # Get indices of the nearest neighbors
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    titles = list(title_to_plot_dict.keys())
    match_count = 0
    for i in indices_of_nearest_neighbors:
        if source_title == titles[i]:
            continue
        if match_count >= k_nearest_neighbors:
            break
        match_count += 1
        print(f"found {match_count} closest match:")
        print(f"distance of: {distances[i]}")
        print(titles[i])

# Usage example:
# title_to_plot_dict = {
#     "The Godfather": "The aging patriarch of an organized crime dynasty...",
#     "The Godfather Part II": "The early life and career of Vito Corleone...",
#     "Goodfellas": "The story of Henry Hill and his life...",
#     "Pulp Fiction": "The lives of two mob hitmen...",
#     "The Shawshank Redemption": "Two imprisoned men bond over...",
#     "Schindler's List": "In German-occupied Poland...",
#     "Forrest Gump": "The presidencies of Kennedy and Johnson...",
#     "The Dark Knight": "When the menace known as the Joker...",
#     "Inception": "A thief who steals corporate secrets...",
#     "Fight Club": "An insomniac office worker...",
#     "The Matrix": "A computer hacker learns...",
# }

source_title = "Phantom Thread"
print_recommendations_from_title(movies_2014, source_title)

Error: Source title not found in the dictionary.
